In [1]:
!pip install tensorflow
!pip install jieba
!pip install nltk
!pip install numpy
!pip install keras

In [2]:
# Get Data
!wget -O sample_data/cmn-eng.zip http://www.manythings.org/anki/cmn-eng.zip
!unzip sample_data/cmn-eng.zip

--2019-01-01 12:19:41--  http://www.manythings.org/anki/cmn-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:30::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 545887 (533K) [application/zip]
Saving to: ‘sample_data/cmn-eng.zip’

sample_data/cmn-eng 100%[===================>] 533.09K  1021KB/s    in 0.5s    

2019-01-01 12:19:42 (1021 KB/s) - ‘sample_data/cmn-eng.zip’ saved [545887/545887]

Archive:  sample_data/cmn-eng.zip
  inflating: cmn.txt                 
  inflating: _about.txt              


In [18]:
!wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.zh.300.vec.gz

--2019-01-01 14:15:59--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.zh.300.vec.gz
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 54.231.235.45
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|54.231.235.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1358817100 (1.3G) [binary/octet-stream]
Saving to: ‘cc.zh.300.vec.gz’

cc.zh.300.vec.gz    100%[===================>]   1.26G  22.1MB/s    in 61s     

2019-01-01 14:17:00 (21.4 MB/s) - ‘cc.zh.300.vec.gz’ saved [1358817100/1358817100]



In [26]:
!head cc.zh.300.vec

2000000 300
， 0.0080 0.0336 0.5720 -0.1105 -0.0200 0.0195 -0.0129 -0.0258 0.0249 -0.0010 0.0182 -0.0074 -0.0288 -0.0116 -0.0355 0.0482 0.0411 0.0008 0.0261 -0.0387 0.0104 -0.0035 0.0715 -0.0001 -0.0397 -0.0353 0.0546 -0.0462 -0.0439 0.0621 -0.0276 0.0151 -0.0528 -0.0016 0.0074 -0.0312 0.0032 -0.0843 -0.0698 -0.0309 0.0178 0.0081 -0.0203 0.0104 -0.1739 -0.0081 -0.0952 -0.0118 0.0211 0.0214 -0.0324 0.0085 -0.0114 -0.4785 -0.0216 0.0469 -0.0026 0.0152 -0.0438 0.0554 -0.0441 -0.0256 -0.0084 -0.0121 -0.0372 -0.0109 0.0704 0.0208 -0.0457 -0.0252 -0.0115 0.0414 0.0078 0.0451 0.0373 0.0149 0.0373 0.0944 0.0333 -0.0020 -0.0143 -0.0006 -0.0178 -0.0050 0.0223 0.0022 0.0922 0.0001 0.0413 0.0115 -0.0239 -0.0202 -0.0142 0.0135 0.0253 -0.0556 -0.0220 -0.0096 0.0073 -0.0090 -0.3952 0.0566 0.0040 -0.0037 0.0147 0.0125 0.0362 0.0277 -0.0226 -0.0104 -0.0305 0.0660 -0.0232 -0.0053 -0.0105 -0.0029 0.0037 0.0066 -0.0339 -0.0758 0.0686 0.0273 -0.0338 -0.0198 -0.0310 -0.0315 0.0456 0.0422 -0.0045 -0.0045 0.16

In [0]:
# Load Chinese Pre-trained Word2vec
# It takes some time to load 
from gensim.models import KeyedVectors

glove_file = "/content/cc.zh.300.vec"

cn_word2vec = KeyedVectors.load_word2vec_format(glove_file)



In [3]:
# Testing chinese Word Embedding
test_cn_word = cn_word2vec['蛇']
print(test_cn_word.shape)

(300,)


In [1]:
# Get pretrained word Embeddings(from fasttext)
# English: https://s3-us-west-1.amazonaws.com/fasttext-vectors/cc.en.300.bin.gz
# Chinese: https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.zh.300.bin.gz
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-01-01 13:39:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-01-01 13:39:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  20.9MB/s    in 42s     

2019-01-01 13:39:50 (19.4 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [0]:
# Load English Pre-trained Word2vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

glove_file = "/content/glove.6B.200d.txt"
tmp_file = get_tmpfile("./test_word2vec.txt")

# call glove2word2vec script
# default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)

en_word2vec = KeyedVectors.load_word2vec_format(tmp_file)


In [5]:
# Testing English Word Embedding
import numpy as np
test_en_word = en_word2vec['going']
print(test_en_word.shape)


(200,)


In [0]:
import tensorflow as tf
import os 
import jieba
import nltk
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
nltk.download('punkt')

# Read Data
data_path = "./cmn.txt"
data = []
if os.path.exists(data_path):
    with open(data_path) as f:
        raw = f.read()
        lines = list(raw.split("\n"))
        lines = lines[0:-1]
        for line in lines:
            if type(line) == type(""):
                per_pair = line.split("\t")
                data.append((per_pair[0], per_pair[1]))
else:
    print("Fail to read data!")
print("Only Load first 10000")
data = data[:10000]

# Tokenizer
cn_lookup = {"<PAD>":0, "<START>":1, "<END>":2,"<UNK>":3}
en_lookup = {"<PAD>":0, "<START>":1, "<END>":2,"<UNK>":3}
cn_idx = 4
en_idx = 4

tokens = []
for en, cn in data:
    cn_token_list = ["<START>"]
    en_token_list = ["<START>"]
    cn_token_list.extend(list(jieba.cut(cn)))
    cn_token_list.extend(["<END>"])
    en_token_list.extend(list(nltk.word_tokenize(en)))
    en_token_list.extend(["<END>"])
    for cn_token in cn_token_list:
        if cn_token not in cn_lookup:
            cn_lookup[cn_token] = cn_idx
            cn_idx += 1
    for en_token in en_token_list:
        if en_token not in en_lookup:
            en_lookup[en_token] = en_idx
            en_idx += 1
    
    tokens.append((en_token_list, cn_token_list))

# Training Parameters
paras = {
        "MAX_EN_LEN" : max([len(token[0]) for token in tokens]),
        "MAX_CN_LEN" : max([len(token[1]) for token in tokens]),
        "EN_VOCAB_SIZE": len(en_lookup),
        "CN_VOCAB_SIZE": len(cn_lookup),
        "NUM_OF_SAMPLES" : len(tokens)
}

# Turn tokens into index
print("Turn Tokens into index")
_tokens = []
for en_token_list, cn_token_list in tokens:
    en_idx_tokens = []
    for token in en_token_list:
        en_idx_tokens.append(en_lookup[token])
    cn_idx_tokens = []
    for token in cn_token_list:
        cn_idx_tokens.append(cn_lookup[token])
    _tokens.append((en_idx_tokens, cn_idx_tokens))

# Initialization of input
print("initialization of input")
encoder_input_data = np.zeros(
        (paras["NUM_OF_SAMPLES"], paras["MAX_EN_LEN"]),
        dtype='float32'
        )
decoder_input_data = np.zeros(
        (paras["NUM_OF_SAMPLES"], paras["MAX_CN_LEN"]),
        dtype='float32'
        )
decoder_target_data = np.zeros(
        (paras["NUM_OF_SAMPLES"], paras["MAX_CN_LEN"], paras["CN_VOCAB_SIZE"] ),
        dtype='float32'
        )
print("shape of encoder_input: {}, decoder_input: {}, output: {}".format(
    encoder_input_data.shape,
    decoder_input_data.shape,
    decoder_target_data.shape
    ))

# Fill in values into np array
print("Fill values into np array")
for idx, (en_idx_tokens, cn_idx_tokens) in enumerate(_tokens):
    for _idx, wd in enumerate(en_idx_tokens):
        encoder_input_data[idx, _idx] = wd
    for _idx, wd in enumerate(cn_idx_tokens):
        decoder_input_data[idx, _idx] = wd
        if _idx > 0:
            decoder_target_data[idx, _idx-1, wd] = 1

# Build pre-trained matrix
EN_EMBEDDING_DIM = 200
CN_EMBEDDING_DIM = 300
cn_embedding_matrix = np.zeros((paras["CN_VOCAB_SIZE"], CN_EMBEDDING_DIM), dtype="float32")
en_embedding_matrix = np.zeros((paras["EN_VOCAB_SIZE"], EN_EMBEDDING_DIM), dtype="float32")

# if we found pretrained vector, then place it in the matrix
for key in en_lookup:
      if key in en_word2vec:
        en_embedding_matrix[en_lookup[key]] = en_word2vec[key]

for key in cn_lookup:
      if key in cn_word2vec:
        cn_embedding_matrix[cn_lookup[key]] = cn_word2vec[key]
            
# Model
# Define
HIDDEN_DIM = 512
# encoder
encoder_embed = Embedding(
    paras["EN_VOCAB_SIZE"], 
    EN_EMBEDDING_DIM, 
    weights=[en_embedding_matrix],
    input_length=paras["MAX_EN_LEN"
], mask_zero=True)
encoder_lstm = LSTM(HIDDEN_DIM, return_state = True)
encoder_inputs = Input(shape = (paras["MAX_EN_LEN"],))
embed_inputs = encoder_embed(encoder_inputs)
_, state_h, state_c = encoder_lstm(embed_inputs)
context_state = [state_h, state_c]
# decoder
decoder_embed = Embedding(
    paras["CN_VOCAB_SIZE"], 
    CN_EMBEDDING_DIM, 
    weights=[cn_embedding_matrix],
    input_length=paras["MAX_CN_LEN"
], mask_zero=True)
decoder_lstm = LSTM(HIDDEN_DIM, return_state = True, return_sequences = True)
decoder_dense = Dense(paras["CN_VOCAB_SIZE"], activation="softmax")
decoder_inputs = Input(shape = (paras["MAX_CN_LEN"], ))
embed_inputs = decoder_embed(decoder_inputs)
outputs, _, _ = decoder_lstm(
    embed_inputs,
    initial_state = context_state
    )
outputs = decoder_dense(outputs)

# Run
print("Start Trianing!")
train_paras = {
    "batch_size": 512,
    "epochs": 50
    }
model = Model([encoder_inputs, decoder_inputs], outputs )
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
    batch_size = train_paras["batch_size"],
    epochs = train_paras["epochs"],
    validation_split = 0.2
    )
model.save('./s.h5')

# Define sampling models
# encoder_model
encoder_model = Model(encoder_inputs, context_state)
# decoder_model
decoder_state_input_h = Input(shape=(HIDDEN_DIM,))
decoder_state_input_c = Input(shape=(HIDDEN_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embed_inputs = decoder_embed(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embed_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
reverse_cn_lookup = {}
for key in cn_lookup:
    reverse_cn_lookup[cn_lookup[key]] = key


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Only Load first 10000
Turn Tokens into index
initialization of input
shape of encoder_input: (10000, 14), decoder_input: (10000, 15), output: (10000, 15, 7002)
Fill values into np array
Start Trianing!
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 18s 2ms/step - loss: 5.0987 - val_loss: 4.8651
Epoch 2/100
8000/8000 [==============================] - 9s 1ms/step - loss: 4.2051 - val_loss: 4.7890
Epoch 3/100
8000/8000 [==============================] - 9s 1ms/step - loss: 4.0057 - val_loss: 4.6279
Epoch 4/100
8000/8000 [==============================] - 9s 1ms/step - loss: 3.8113 - val_loss: 4.5735
Epoch 5/100
8000/8000 [==============================] - 9s 1ms/step - loss: 3.6465 - val_loss: 4.5534
Epoch 6/100
8000/8000 [==============================] - 9s 1ms/step - loss: 3.4892 - val_loss: 4.3948
Epoch 7/100
8000/800

In [0]:
def translate(input_seq):
    context_states = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, paras["MAX_CN_LEN"]), dtype="float32")
    target_seq[0, 0] = cn_lookup["<START>"]
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
                [target_seq] + context_states)
        token_idx = np.argmax(output_tokens[0, -1, :])
        predict_token = reverse_cn_lookup[token_idx]
        decoded_sentence += predict_token
        if predict_token == "<END>" or len(decoded_sentence) > paras["MAX_CN_LEN"]:
            stop_condition = True
        target_seq = np.zeros((1, paras["MAX_CN_LEN"]), dtype="float32")
        target_seq[0, 0] = token_idx
        context_states = [h, c]
    return decoded_sentence

In [0]:
# Run
# 4077, 2122, 3336, 1463, 8954, 7164, 3491, 4495, 5097, 118
chosen = [4077, 2122, 3336, 1463, 8954, 7164, 3491, 4495, 5097, 118 ]
for idx in chosen:
    fed = encoder_input_data[idx:idx+1]
      
    print(tokens[idx])
    sent = translate(fed)
    print(sent)



In [0]:
# Save result to output.txt
with open('output.txt', 'w') as f:
    for idx in chosen:
        fed = encoder_input_data[idx:idx+1]
        input_sent = tokens[idx]
        input_sent = input_sent[0]
        decoded_sent = translate(fed)
        f.write("Input sentence\t" + input_sent)
        f.write("Decoded sentence\t" + decoded_sent)
        f.write("-")
